## Домашнее задание к лекции "Основы веб-скрапинга и работы с API"

### Задание 1.

#### Обязательная часть
> Будем парсить страницу со свежими новостям на habr.com/ru/all/. \
Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например: \
KEYWORDS = ['python', 'парсинг'] \
Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы) \
В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>

#### Дополнительная часть (необязательная)
> Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком. \
Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы. \
Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст_статьи>

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [13]:
# список ключевых слов
KEYWORDS = ['samsung', 'python', 'java']
URL = 'https://m.habr.com/ru/news/page'
URLs = 'https://m.habr.com'
# ограничиваем глубину выдачи в 5 страниц
pages = range(1, 10)
all_posts = pd.DataFrame()

In [14]:
for page in pages:
    req = requests.get(URL+str(page))
    soup = BeautifulSoup(req.text, 'html.parser')
    # извлекаем посты
    posts = soup.find_all(class_='tm-article-snippet')
    time.sleep(0.3)
    for post in posts:
        date = pd.to_datetime(post.find(class_='tm-article-snippet__datetime-published').find('time').get('datetime'), dayfirst=True).date()
        title = post.find(class_='tm-article-snippet__title tm-article-snippet__title_h2').text    
        link = URLs+post.find(class_='tm-article-snippet__title-link').get('href')
        for keyword in KEYWORDS: # ищем вхождение каждого выбраного слова
            if keyword.lower() in title.lower().split(): # в тайтле, сохраняем при вхождении любого
                row = {'date': date, 'title': title, 'link': link}
                all_posts = pd.concat([all_posts, pd.DataFrame([row])])
                continue                                 # при первом найденном выходим из проверки поста, не перебираем всё
all_posts.head()

,date,title,link
0,2021-03-25,Samsung представила модуль памяти DDR5 на 512 ГБ,https://m.habr.com/ru/news/t/548950/
0,2021-03-17,Samsung предупредила о возможном отказе от Gal...,https://m.habr.com/ru/news/t/547596/
0,2021-03-17,Huawei начнет брать с Apple и Samsung лицензио...,https://m.habr.com/ru/news/t/547530/


### Задание 2.

#### Обязательная часть
> Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода:
EMAIL = [xxx@x.ru, yyy@y.com] \
В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки> \
Подсказка: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы. 

#### Дополнительная часть (необязательная)
> Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте. \
Документация к API VK: https://vk.com/dev/methods , вам поможет метод wall.get \
GROUP = 'netology'  \
TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ \
В итоге должен формироваться датафрейм со столбцами: <дата поста> - <текст поста> 

In [664]:
EMAILS = ['1048888@rambler.ru', 'info@tile.delivery', '123@ya.ru']
url = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'
headers = {'Vaar-Version': '0', 'Vaar-Header-App-Product-Name': 'hackcheck-web-avast', 'Vaar-Header-App-Build-Version': '1.0.0'}
all_allerts = pd.DataFrame()

In [659]:
email_request = {'emailAddresses':EMAILS}
avast_response = requests.post(url, json = email_request, headers = headers)
union_request = avast_response.json()

In [665]:
for key, values in union_request['summary'].items():
    for key_jun, value in values.items():
        for i in value:
            email = key
            i = str(i)
            date = pd.to_datetime(union_request['breaches'][i]['publishDate'], dayfirst=False).date()
            source = union_request['breaches'][i]['site']
            description = union_request['breaches'][i]['description']
            row = {'email': email, 'date': date, 'source': source, 'description': description}
            all_allerts = pd.concat([all_allerts, pd.DataFrame([row])])
all_allerts.head(10)

,email,date,source,description
0,1048888@rambler.ru,2016-10-23,rambler.ru,"On February 17, 2012, Russian email provider R..."
0,123@ya.ru,2020-12-10,raver.fm,"In November 2020, a collection of over 23,000 ..."
0,123@ya.ru,2021-01-21,bonobos.com,"At an unconfirmed date, the men's clothing ret..."
0,123@ya.ru,2020-12-10,light.muztorg.ru,"In November 2020, a collection of over 23,000 ..."
0,123@ya.ru,2020-12-17,btravel.ru,"In November 2020, a collection of over 23,000 ..."
0,123@ya.ru,2020-12-03,flexicon.uk.com,"In November 2020, a collection of over 23,000 ..."
0,123@ya.ru,2020-12-10,techlinedisk.ru,"In November 2020, a collection of over 23,000 ..."
0,123@ya.ru,2020-12-10,zildjian.ru,"In November 2020, a collection of over 23,000 ..."
0,123@ya.ru,2020-12-10,gkist.su,"In November 2020, a collection of over 23,000 ..."
0,123@ya.ru,2020-12-10,tltmail.ru,"In November 2020, a collection of over 23,000 ..."
